In [99]:
import ee
import geemap
import os
import numpy as np

In [130]:
#Create Map and center on Manauas EC Flux Tower coordinates
Map = geemap.Map(center = (-2.609097222, -60.20929722), zoom = 10)
brma2_point = ee.Geometry.Point(-60.20929722, -2.609097222)
brma2_region = ee.Geometry.Rectangle(-60.50, -2.85, -60.00, -2.35)
Map

Map(center=[-2.609097222, -60.20929722], controls=(WidgetControl(options=['position'], widget=HBox(children=(T…

In [131]:
Map.addLayer(brma2_region, {}, 'Manauas Region')

In [132]:
def addNDRE(image):
    ndre = image.normalizedDifference(['B8', 'B5']).rename('NDRE')
    return image.addBands(ndre)
ndre_params = ({'min': 0.0,'max': 1.0, 'palette' : 
               ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301']})

In [133]:
#Load sentinel-2 data and clip to brma_2; set visual params
s2 = (ee.ImageCollection('COPERNICUS/S2')
     .filterDate('2016-01-01', '2016-12-31')
     .filterBounds(brma2_region)
     .map(addNDRE))
visParams = {'bands' : ['B4', 'B3', 'B2'], 'max': 3000}

In [134]:
#Handy list of Sentinel-2 property names
#Note; .get() function should grab things within each property
properties = s2.first().propertyNames().getInfo()
print(properties)

['DATATAKE_IDENTIFIER', 'SPACECRAFT_NAME', 'FORMAT_CORRECTNESS_FLAG', 'system:id', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A', 'MEAN_SOLAR_AZIMUTH_ANGLE', 'system:footprint', 'SOLAR_IRRADIANCE_B12', 'system:version', 'SOLAR_IRRADIANCE_B10', 'SOLAR_IRRADIANCE_B11', 'GENERATION_TIME', 'SOLAR_IRRADIANCE_B8A', 'SENSOR_QUALITY_FLAG', 'CLOUD_COVERAGE_ASSESSMENT', 'system:time_end', 'system:time_start', 'DATASTRIP_ID', 'PROCESSING_BASELINE', 'SENSING_ORBIT_NUMBER', 'GEOMETRIC_QUALITY_FLAG', 'SENSING_ORBIT_DIRECTION', 'GRANULE_ID', 'REFLECTANCE_CONVERSION_CORRECTION', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8', 'DATATAKE_TYPE', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B9', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B6', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B7', 'RADIOMETRIC_QUALITY_FLAG', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B4', 'MEAN_INCIDENCE_ZENITH_ANGLE_B1', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B5', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B2', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B3', 'MEAN_INCIDENCE_ZENITH_ANGLE_B5', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B1', 'MEAN_INCIDEN

In [138]:
#From https://gis.stackexchange.com/questions/345598/mapping-simple-function-to-print-date-and-time-stamp-on-google-earth-engine-pyth
dates = (ee.List(s2
    .aggregate_array('system:time_start'))
    .map(lambda time_start: ee.Date(time_start).format('Y-MM-dd'))
         .getInfo())

dates_list = []

for date in dates:
    dates_list.append(date)

print(dates_list)
print(len(dates_list))

['2016-03-22', '2016-03-22', '2016-04-11', '2016-04-11', '2016-04-21', '2016-04-21', '2016-05-01', '2016-05-01', '2016-05-11', '2016-05-11', '2016-05-21', '2016-05-21', '2016-06-10', '2016-06-10', '2016-06-20', '2016-06-20', '2016-06-30', '2016-06-30', '2016-07-10', '2016-07-20', '2016-07-30', '2016-08-19', '2016-08-19', '2016-09-08', '2016-09-08', '2016-09-28', '2016-09-28', '2016-10-08', '2016-10-08', '2016-10-18', '2016-10-18', '2016-10-28', '2016-10-28', '2016-11-07', '2016-11-07', '2016-11-17', '2016-11-17', '2016-11-27', '2016-11-27', '2016-12-07', '2016-12-17', '2016-12-27']
42


In [148]:
#Attempting to print the values associated with each band at a point
#Honestly not sure how to get vaues for a pixel on python
s2 = (ee.ImageCollection('COPERNICUS/S2')
     .filterDate('2016-01-01', '2016-12-31')
     .filterBounds(brma2_point)
     .map(addNDRE))

band_values = []

bands = s2.aggregate_array('system:band').getInfo()

for band in bands:
    band_arr = s2.get(band).getInfo()
    band_values.append(band_arr)

print(band_values)
print(len(band_values))

[]
0


In [160]:
#A second attempt
#Also, it doesn't work (-:
point = ee.Geometry.Point(-60.20929722, -2.609097222)

product_name = s2
start = '2016-01-01'
end = '2016-12-31'

# Obtain image collection for all images within query dates
coll = ee.ImageCollection(product_name).filterDate(start, end)

images = [item.get('id') for item in coll.getInfo().get('features')]

store = []

for image in images:
    im = ee.Image(image)
    data = (im.select('NDRE')
            .reduceRegion(ee.Reducer.first(), point , 1)
            .get('NDRE'))
    store.append(data.getInfo())

print(store)

EEException: Image.select: Pattern 'NDRE' did not match any bands.